In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd '/content/drive/MyDrive/브런치추천/'

/content/drive/MyDrive/브런치추천


In [3]:
data = pd.read_csv('processed_data/data.csv')

In [4]:
data.head(1)

,dt,hr,user_id,article_id,writer_id
0,20190111,5,#ad379c552d279aa0133666cbca5ed0ad,387,@cosmos-j


In [5]:
metadata = pd.read_json('data/metadata.json',lines=True)

In [6]:
print(metadata.shape)
print(data['article_id'].nunique())

(643104, 9)
3185


In [7]:
metadata.head(1)

,magazine_id,user_id,title,keyword_list,display_url,sub_title,reg_ts,article_id,id
0,8982,@bookdb,"사진으로 옮기기에도 아까운, 리치필드 국립공원","[여행, 호주, 국립공원]",https://brunch.co.kr/@bookdb/782,세상 어디에도 없는 호주 Top 10,1474944427000,782,@bookdb_782


In [8]:
tmp = data[['article_id','writer_id']].drop_duplicates().reset_index(drop=True)
tmp = tmp.rename(columns={'writer_id':'user_id'})

In [9]:
metadata = pd.merge(metadata,tmp,on=['user_id','article_id'])

In [10]:
metadata.head(1)

,magazine_id,user_id,title,keyword_list,display_url,sub_title,reg_ts,article_id,id
0,28849,@elang8151,출간제의 받았습니다.,"[메일, 출판사]",https://brunch.co.kr/@elang8151/229,미안하지만 아직 학생입니다.,1539405177000,229,@elang8151_229


In [11]:
print(metadata.shape)

(149751, 9)


In [12]:
metadata.to_json('processed_data/metadata.json')